In [1]:
from __future__ import annotations

import time
from PersistentHomology.PersHom2 import PersHom2
from PersistentHomology.PersHom1 import PersHom1
import itertools
from Complexes.Simplex import Simplex
from Complexes.FilteredSimplicialComplex import FilteredSimplicialComplex

from plotnine import ggplot, aes, geom_line, labs, theme, xlab, ylab, scale_x_continuous
import pandas as pd
import numpy as np
import random


In [2]:
from typing import List
from Complexes.VR.Expansion.Incremental import Incremental
from Complexes.VR.Skeleton.Sklearn import Sklearn


def generate_n_sphere(n: int) -> FilteredSimplicialComplex:
    filt_comp = FilteredSimplicialComplex(check_valid=True)
    vertices = list(range(n+2))
    for l in range(1, n+2):
        for elem in itertools.combinations(vertices, l):
            filt_comp.add_simplex(Simplex(set(elem)), l)
    return filt_comp

def sample_noisy_circle(point_count: int, radius: float = 1, radius_noise: float = None) -> List[np.ndarray]:
    if radius_noise is None:
        radius_noise = radius / 10
    points = []
    for _ in range(point_count):
        r = radius + ((((random.random() ** 0.5) * 2) - 1) * radius_noise)
        angle = 2 * np.pi * random.random()
        points.append(np.array([r * np.cos(angle), r * np.sin(angle)]))
    return points


def d(x: np.ndarray, y: np.ndarray) -> float:
    if x.size != y.size:
        raise ValueError("Sizes of x and y do not match.")
    total = 0
    for xx, yy in zip(x, y):
        total += (xx - yy) ** 2
    return total ** 0.5

class VR(Sklearn, Incremental):
    pass

def generate_noisy_sphere_complex(n: int, epsilon: float = 0.01) -> FilteredSimplicialComplex:
    run = VR(sample_noisy_circle(n), epsilon, d)
    run.compute_skeleton()
    run.compute_expansion(10)
    return run.get_complex()


First we look at 1 vs 2.

In [55]:
results = {
    "n": list(),
    "load_1": list(),
    "load_2": list(),
    "compute_1": list(),
    "compute_2": list()
}

for i in range(1, 10):
    fc = generate_n_sphere(i)
    print(f"Simplicial complex build with {fc.size} simplices.")
    results["n"].append(i)

    start = time.perf_counter()
    pers_hom_1 = PersHom1(fc)
    end = time.perf_counter()
    results["load_1"].append(end - start)

    start = time.perf_counter()
    pers_hom_2 = PersHom2(fc)
    end = time.perf_counter()
    results["load_2"].append(end - start)

    start = time.perf_counter()
    pers_hom_1.compute()
    end = time.perf_counter()
    results["compute_1"].append(end - start)

    start = time.perf_counter()
    pers_hom_2.compute()
    end = time.perf_counter()
    results["compute_2"].append(end - start)

Simplicial complex build with 6 simplices.
Simplicial complex build with 14 simplices.
Simplicial complex build with 30 simplices.
Simplicial complex build with 62 simplices.
Simplicial complex build with 126 simplices.
Simplicial complex build with 254 simplices.
Simplicial complex build with 510 simplices.
Simplicial complex build with 1022 simplices.
Simplicial complex build with 2046 simplices.


In [56]:
x = results["n"]
y1 = results["compute_1"]
y2 = results["compute_2"]
df = pd.DataFrame(zip(x, y1, y2), columns=["n", "S", "SS"])
df = pd.melt(df, id_vars="n")

p = (
        ggplot(df)
        + aes(x="n")
        + geom_line(aes(y="value", linetype="variable"), size=1)
        + labs(linetype="Algorithm")
        + xlab("$n$")
        + ylab("Time (s)")
        + theme(figure_size=(8, 4), dpi=300)
        + scale_x_continuous(breaks=[1,2,3,4,5,6,7,8,9])
)
p.save("2-1v2-compute.pdf", "pdf")

C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:719: PlotnineWarning: Saving 8 x 4 in image.
C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:722: PlotnineWarning: Filename: 2-1v2-compute.pdf


In [57]:
x = results["n"]
y1 = results["load_1"]
y2 = results["load_2"]
df = pd.DataFrame(zip(x, y1, y2), columns=["n", "S", "SS"])
df = pd.melt(df, id_vars="n")

p = (
        ggplot(df)
        + aes(x="n")
        + geom_line(aes(y="value", linetype="variable"), size=1)
        + labs(linetype="Algorithm")
        + xlab("$n$")
        + ylab("Time (s)")
        + theme(figure_size=(8, 4), dpi=300)
        + scale_x_continuous(breaks=[1,2,3,4,5,6,7,8,9])
)
p.save("2-1v2-load.pdf", "pdf")

C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:719: PlotnineWarning: Saving 8 x 4 in image.
C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:722: PlotnineWarning: Filename: 2-1v2-load.pdf


Let's do the same test but with a noisy circle...

In [58]:
results = {
    "n": list(),
    "load_1": list(),
    "load_2": list(),
    "compute_1": list(),
    "compute_2": list()
}

for i in np.arange(100, 1501, 50):
    fc = generate_noisy_sphere_complex(i)
    print(f"Simplicial complex build with {fc.size} simplices.")
    results["n"].append(i)

    start = time.perf_counter()
    pers_hom_1 = PersHom1(fc)
    end = time.perf_counter()
    results["load_1"].append(end - start)

    start = time.perf_counter()
    pers_hom_2 = PersHom2(fc)
    end = time.perf_counter()
    results["load_2"].append(end - start)

    start = time.perf_counter()
    pers_hom_1.compute()
    end = time.perf_counter()
    results["compute_1"].append(end - start)

    start = time.perf_counter()
    pers_hom_2.compute()
    end = time.perf_counter()
    results["compute_2"].append(end - start)

Simplicial complex build with 102 simplices.
Simplicial complex build with 154 simplices.
Simplicial complex build with 208 simplices.
Simplicial complex build with 257 simplices.
Simplicial complex build with 312 simplices.
Simplicial complex build with 362 simplices.
Simplicial complex build with 423 simplices.
Simplicial complex build with 492 simplices.
Simplicial complex build with 535 simplices.
Simplicial complex build with 604 simplices.
Simplicial complex build with 653 simplices.
Simplicial complex build with 709 simplices.
Simplicial complex build with 776 simplices.
Simplicial complex build with 832 simplices.
Simplicial complex build with 909 simplices.
Simplicial complex build with 958 simplices.
Simplicial complex build with 1041 simplices.
Simplicial complex build with 1093 simplices.
Simplicial complex build with 1164 simplices.
Simplicial complex build with 1208 simplices.
Simplicial complex build with 1294 simplices.
Simplicial complex build with 1351 simplices.
Simp

In [60]:
x = results["n"]
y1 = results["compute_1"]
y2 = results["compute_2"]
df = pd.DataFrame(zip(x, y1, y2), columns=["n", "S", "SS"])
df = pd.melt(df, id_vars="n")

p = (
        ggplot(df)
        + aes(x="n")
        + geom_line(aes(y="value", linetype="variable"), size=1)
        + labs(linetype="Algorithm")
        + xlab("Number of simplices")
        + ylab("Time (s)")
        + theme(figure_size=(8, 4), dpi=300)
        + scale_x_continuous(breaks=[100, 300, 500, 700, 900, 1100, 1300, 1500])
)
p.save("2-1v2-circ-compute.pdf", "pdf")

C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:719: PlotnineWarning: Saving 8 x 4 in image.
C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:722: PlotnineWarning: Filename: 2-1v2-circ-compute.pdf


In [27]:
core_complex = generate_noisy_sphere_complex(1000, 0.06)

In [28]:
complexes = []
for ep in np.arange(0.01, 0.06 + 0.0005, 0.0005): # use 0.0005
    complexes.append(core_complex.cap(ep))

Now 2 vs 3.

In [10]:
from PersistentHomology.PersHom3 import PersHom3

results = {
    "n": list(),
    "compute_2": list(),
    "compute_3": list()
}

for fc in complexes:
    results["n"].append(fc.size)

    pers_hom_2 = PersHom2(fc)
    pers_hom_3 = PersHom3(fc)

    start = time.perf_counter()
    pers_hom_2.compute()
    end = time.perf_counter()
    results["compute_2"].append(end - start)

    start = time.perf_counter()
    pers_hom_3.compute()
    end = time.perf_counter()
    results["compute_3"].append(end - start)

KeyboardInterrupt: 

In [ ]:
x = results["n"]
y1 = results["compute_2"]
y2 = results["compute_3"]
df = pd.DataFrame(zip(x, y1, y2), columns=["n", "SS", "SSR"])
df = pd.melt(df, id_vars="n")

p = (
        ggplot(df)
        + aes(x="n")
        + geom_line(aes(y="value", linetype="variable"), size=1)
        + labs(linetype="Algorithm")
        + xlab("Number of simplices")
        + ylab("Time (s)")
        + theme(figure_size=(8, 4), dpi=300)
)
p.save("2-2v3-circ-compute.pdf", "pdf")

3 vs 4

In [32]:
from PersistentHomology.PersHom3 import PersHom3
from PersistentHomology.PersHom4 import PersHom4


results = {
    "n": list(),
    "compute_3": list(),
    "compute_4": list()
}

for i, fc in enumerate(complexes):
    print(i)
    results["n"].append(fc.size)

    pers_hom_3 = PersHom3(fc)
    pers_hom_4 = PersHom4(fc)

    start = time.perf_counter()
    pers_hom_3.compute()
    end = time.perf_counter()
    results["compute_3"].append(end - start)
    print(f"compute_3: {round(end - start, 3)}")

    start = time.perf_counter()
    pers_hom_4.compute()
    end = time.perf_counter()
    results["compute_4"].append(end - start)
    print(f"compute_4: {round(end - start, 3)}")
    print()

0
compute_3: 0.002
compute_4: 0.001

1
compute_3: 0.003
compute_4: 0.001

2
compute_3: 0.002
compute_4: 0.001

3
compute_3: 0.002
compute_4: 0.001

4
compute_3: 0.002
compute_4: 0.002

5
compute_3: 0.002
compute_4: 0.002

6
compute_3: 0.002
compute_4: 0.001

7
compute_3: 0.002
compute_4: 0.002

8
compute_3: 0.003
compute_4: 0.002

9
compute_3: 0.002
compute_4: 0.002

10
compute_3: 0.003
compute_4: 0.002

11
compute_3: 0.003
compute_4: 0.002

12
compute_3: 0.003
compute_4: 0.002

13
compute_3: 0.003
compute_4: 0.002

14
compute_3: 0.003
compute_4: 0.002

15
compute_3: 0.003
compute_4: 0.002

16
compute_3: 0.004
compute_4: 0.003

17
compute_3: 0.004
compute_4: 0.003

18
compute_3: 0.005
compute_4: 0.003

19
compute_3: 0.005
compute_4: 0.004

20
compute_3: 0.005
compute_4: 0.003

21
compute_3: 0.006
compute_4: 0.003

22
compute_3: 0.005
compute_4: 0.004

23
compute_3: 0.006
compute_4: 0.004

24
compute_3: 0.006
compute_4: 0.004

25
compute_3: 0.007
compute_4: 0.006

26
compute_3: 0.439
co

In [54]:
x = results["n"]
y1 = results["compute_3"]
y2 = results["compute_4"]

x_aug = []
y1_aug = []
y2_aug = []

for i, (a,b,c) in enumerate(zip(x, y1, y2)):
    print(i)
    print(b)
    print(c)
    print()


indices = [26,34, 40, 45, 49, 55, 52,  67, 70, 71,76, 77, 78,81, 82, 83, 85,86, 87,88, 89, 90, 98, 99]

for i in range(len(x)):
    if i not in indices:
        x_aug.append(x[i])
        y1_aug.append(y1[i])
        y2_aug.append(y2[i])


df = pd.DataFrame(zip(x_aug, y1_aug, y2_aug), columns=["n", "SSR", "SSRCoh"])
df = pd.melt(df, id_vars="n")

p = (
        ggplot(df)
        + aes(x="n")
        + geom_line(aes(y="value", linetype="variable"), size=1)
        + labs(linetype="Algorithm")
        + xlab("Number of simplices")
        + ylab("Time (s)")
        + theme(figure_size=(8, 4), dpi=300)
)
p.save("2-3v4-circ-compute.pdf", "pdf")

0
0.002163599999903454
0.0013407999999799358

1
0.0029603999998926156
0.0012601000000813656

2
0.0016821000001527864
0.001486200000044846

3
0.002416600000060498
0.001496999999972104

4
0.0021782000001167034
0.0017106000000239874

5
0.002277800000001662
0.00177680000001601

6
0.002243200000066281
0.0014734999999745924

7
0.002330700000129582
0.001559999999926731

8
0.0028064000000540545
0.0017533000000184984

9
0.0024895999999898777
0.0021888000001126784

10
0.00257129999999961
0.0018740999998954067

11
0.0026417000001401902
0.0019560000000637956

12
0.002765899999985777
0.002083000000084212

13
0.002898700000059762
0.0021686999998564715

14
0.0031238999999914085
0.0022968000000673783

15
0.003467399999863119
0.00245279999990089

16
0.00428909999982352
0.0026740000000700093

17
0.00366159999998672
0.002763100000038321

18
0.004561599999988175
0.003214700000171433

19
0.0052325999999993655
0.004153600000108781

20
0.005004800000051546
0.003142900000057125

21
0.005538900000146896
0.0034

C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:719: PlotnineWarning: Saving 8 x 4 in image.
C:\Users\napie\anaconda3\lib\site-packages\plotnine\ggplot.py:722: PlotnineWarning: Filename: 2-3v4-circ-compute.pdf


In [ ]:
from PersistentHomology.PersHom3 import PersHom3
from PersistentHomology.PersHom4 import PersHom4


pers_hom_3 = PersHom3(complexes[0])
pers_hom_4 = PersHom4(complexes[0])

pers_hom_3.compute()
pers_hom_4.compute()



In [ ]:
print(pers_hom_3.get_pers_diag() == pers_hom_4.get_pers_diag())